In [1]:
## Note: please make sure navitron-crons is already installed in venv
## https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import plotnine

import navitron_crons.connections as connections
import navitron_crons.cli_core as cli_core

CONN = connections.MongoConnection(cli_core.CONFIG)
%matplotlib inline

/Users/jpurcell/source/NavitronEve/jvenv_navitron_mac/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
